In [3]:
import numpy as np
from sklearn import linear_model
import pandas as pd
from sklearn.model_selection import train_test_split

A análise em questão visa identificar uma relação linear de proporcionalidade entre os valores de saneamento básico e IDH com os dados de mortalidade por tuberculose. Para isso, iremos utilizar as seguintes informações:
- Proportion of population using at least basic sanitation services
- Proportion of population using at least basic drinking water services
- IDH
- Tuberculosis
A partir desses dados, iremos realizar uma regressão linear em que Y representa os valores de mortalidade por tuberculose e X os indicadores restantes.
Com os coeficientes definidos, será possível perceber qual dos indicadores possui maior relevância na determinação das mortes por tuberculose. Isto é, aquele que acompanha um coeficiente de maior valor absoluto, possui maior influência na mortalidade.

In [4]:
IDH_data = pd.read_csv('../data/processed/IDH_data.csv')
sanitation_data = pd.read_csv('../data/processed/sanitation_data.csv')
country_data = pd.read_csv('../data/processed/countries_data.csv')
mortality_data = pd.read_csv('../data/processed/mortality_data.csv')

Primeiramente devemos tratar dos dados. É necessário verificar que todos os dados utilizados estejam completos, ou seja, que haja valores não nulos para todos os indicadores analisados. Após isso, basta coletar as informações de cada país nos anos disponíveis.

In [5]:
mortality_data = mortality_data.loc[mortality_data['year'] >= 2000]

IDH_data = IDH_data.loc[IDH_data['year'] >= 2000]

x = []
y = []

for country in country_data['Country']:

    m_data = mortality_data.loc[(mortality_data['country'] == country)]

    idh_data = IDH_data.loc[IDH_data['country'] == country]
    
    soap_data = sanitation_data.loc[(sanitation_data['country'] == country) & (sanitation_data['INDICATOR'] == 'WS_PPL_S-ALB')]
    #Proportion of population using at least basic sanitation services

    water_data = sanitation_data.loc[(sanitation_data['country'] == country) & (sanitation_data['INDICATOR'] == 'WS_PPL_W-ALB')]
    #Proportion of population using at least basic drinking water services

    validated_years = [] #vetor que salva os anos que contém valores de soap e water

    for year in m_data['year']:

        if ((year in soap_data['year'].to_numpy()) & (year in water_data['year'].to_numpy()) & (year in idh_data['year'].to_numpy())):

            validated_years.append(year)

    #print(validated_years)
    for year in validated_years:
        
        idh_value = idh_data.loc[idh_data['year'] == year]
        soap_value = soap_data.loc[soap_data['year'] == year]
        water_value = water_data.loc[water_data['year'] == year]

        row = [idh_value['HDI_VALUE'].values[0], soap_value['OBS_VALUE'].values[0]/100, water_value['OBS_VALUE'].values[0]/100]
        #dividimos por cem pois está em porcentagem
        x.append(row)

        mortality_value = m_data.loc[m_data['year'] == year] 
        y.append(mortality_value['Tuberculosis'].values[0])

Agora, vamos inserir os dados no modelo e analisar sua acurácia

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

regr = linear_model.LinearRegression()
regr.fit(x_train, y_train)

print('Coeficientes: ', regr.coef_)
print('Intersecção: ', regr.intercept_)

print('Variância: ', regr.score(x_test, y_test))

Coeficientes:  [-4.61155195 -2.38153482 -1.91703712]
Intersecção:  8.311102119916312
Variância:  0.5460660701790819
